In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
from datetime import datetime
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
links = [
"destinasi",
"fokus-dun",
"komentar",
"nasional",
"sukan",
"bisnes",
"didik",
"english"]

hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 



In [7]:
for i in tqdm(links):
    
    x = 1

    while True:

        while True:
            try:
                r = requests.get(f'https://www.melakahariini.my/category/{i}/page/{x}',headers = headers)
                break
            except Exception as e:
                print(e)
                time.sleep(1.0)

        soup = BeautifulSoup(r.text, "lxml")

        div = soup.find_all('h2', attrs = {"class":"entry-title"})

        if len(div) == 0:
            break

        for link in soup.find_all('h2', attrs = {"class":"entry-title"}):

            href = link.find('a').get('href')

            hrefs.append(href)


        x = x + 1



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [19:47<00:00, 148.39s/it]


In [9]:
hrefs = list(set(hrefs))

In [10]:
with open('melakahariini-link.json', 'a') as f:
    json.dump(hrefs, f)

In [3]:
len(hrefs)

13208

In [4]:
with open('melakahariini-link.json') as fopen:
    hrefs = json.load(fopen)

In [5]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="entry-title entry-title-large").text
        h = soup.find('div', class_="post-content")
        p = [p.text for p in h.find_all("p") if len(p.text.split()) > 20]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [6]:
max_workers = 50

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('melakahariini.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13208/13208 [20:42<00:00, 10.63it/s]
